In [10]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.llms import Ollama
from langchain_community.agent_toolkits import create_sql_agent

llm = Ollama(model="phi3")
llm.invoke("How well do you know SQL and can write SQL queries?")       #test
db = SQLDatabase.from_uri("sqlite:///chinook.db")


Sql

### Langchain agent executor

In [11]:

agent_executor = create_sql_agent(llm, db=db, verbose=False)
# agent_executor.invoke(
#     "List the total sales per country. Which country's customers spent the most?"
# )

In [12]:
agent_executor.invoke(
    "How many artists are there?"
)

{'input': 'How many artists are there?',
 'output': 'There are 275 artists in the database.'}

## Alejandro tutorial

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities.sql_database import SQLDatabase


### Subchain

#### Prompt template

In [28]:
template = """
Based on the schema below, write an SQL query that would answer the user's question:
{schema}

*Make sure to output only a runnable SQL Query, do not output any other information. The last character in the output must be a ;

Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

prompt.format(schema="my schema", question="How many users are there?")

"Human: \nBased on the schema below, write an SQL query that would answer the user's question:\nmy schema\n\n*Make sure to output only a runnable SQL Query, do not output any other information. The last character in the output must be a ;\n\nQuestion: How many users are there?\nSQL Query:\n"

#### Load SQL Database

In [29]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

db.run("SELECT * FROM Album Limit 5")                       #test db is loaded correctly, test query


"[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]"

#### DB schema to python object

In [30]:
def get_schema(_):
    return db.get_table_info()

get_schema(None)

'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

#### Create SQL Chain

In [31]:
from langchain_core.output_parsers import StrOutputParser               #Getting SQL squery, then running query in db, so string is required
from langchain_core.runnables import RunnablePassthrough                #allows us to pass through a function so our chain can use it


In [32]:
llm = Ollama(model="phi3")

#### Chain
- RunnablePassthrough.assign(schema=get_schema) --> allow langchain to access db 
- prompt --> Prompt telling llm what to do
- llm --> call LLM
- REMOVED: llm.bind(stop="\nSQL Result: ") --> ensure that there is a stop marker for the output of your LLM
- StrOutputParser()  --> ensure output from chain is a string

In [40]:
schema = db.get_table_info()

In [42]:
sql_chain = RunnablePassthrough.assign(schema=db.get_table_info) | prompt | llm | StrOutputParser()                 #chain


In [37]:
query = sql_chain.invoke({"question": "How many artists are there?"})

In [39]:
query

' SELECT COUNT(DISTINCT Composer) FROM Track;'

### Full chain

In [22]:
template_meta = """
Based on the question, sql query, sql response, and table schema below, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}
"""

prompt = ChatPromptTemplate.from_template(template_meta)

In [23]:
def run_query(query):
    """
    Takes query and runs query in database
    """
    return db.run(query)

In [24]:
query
run_query(query)

'[(853,)]'

In [25]:
full_chain = RunnablePassthrough.assign(query=sql_chain).assign(schema=get_schema, response= lambda variables: run_query(variables["query"])) | prompt | llm 

In [26]:
full_chain.invoke({"question": "How many artists are there?"})

' The SQL query provided correctly returns the number of distinct artists from the `Track` table. However, according to the sample data given in the table, there are only 3 unique composers (Angus Young, Malcolm Young, Brian Johnson for track with ID 1; U. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann for track with ID 2; F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman for track with ID 3).\n\nGiven the sample data:\n\n```sql\nSELECT COUNT(DISTINCT Composer) AS NumberOfArtists FROM Track;\n```\n\nSQL Response should be: `[(3,)]`'